In [1]:
# -*- coding:utf8 -*-
'''
Jessie
'''

import jieba
import pickle
# import cPickle
import pandas as pd
import numpy as np
import time
from itertools import islice
# from sklearn.model_selection import train_test_split
import json
import os
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Input,Flatten,Dense, concatenate
from keras.models import Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.optimizers import Adagrad,RMSprop, Adam,SGD
from keras.callbacks import EarlyStopping
import h5sparse
import gensim

/home/jeshe/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import json


def init():
    f = open('./CAIL_data/law.txt', 'r', encoding = 'utf8')
    law = {}
    lawname = {}
    line = f.readline()
    while line:
        lawname[len(law)] = line.strip()
        law[line.strip()] = len(law)
        line = f.readline()
    f.close()


    f = open('./CAIL_data/accu.txt', 'r', encoding = 'utf8')
    accu = {}
    accuname = {}
    line = f.readline()
    while line:
        accuname[len(accu)] = line.strip()
        accu[line.strip()] = len(accu)
        line = f.readline()
    f.close()


    return law, accu, lawname, accuname


law, accu, lawname, accuname = init()

In [3]:
def getClassNum(kind):
    global law
    global accu

    if kind == 'law':
        return len(law)
    if kind == 'accu':
        return len(accu)

def getName(index, kind):
    global lawname
    global accuname
    if kind == 'law':
        return lawname[index]

    if kind == 'accu':
        return accuname[index]


def gettime(time):
    #将刑期用分类模型来做
    v = int(time['imprisonment'])

    if time['death_penalty']:
        return 0
    if time['life_imprisonment']:
        return 1
    elif v > 10 * 12:
        return 2
    elif v > 7 * 12:
        return 3
    elif v > 5 * 12:
        return 4
    elif v > 3 * 12:
        return 5
    elif v > 2 * 12:
        return 6
    elif v > 1 * 12:
        return 7
    else:
        return 8


def getlabel(d, kind):
    global law
    global accu    
    # 做单标签
    if kind == 'law':
    # 返回多个类的第一个
        return law[str(d['meta']['relevant_articles'][0])]
    if kind == 'accu':
        return accu[d['meta']['accusation'][0]]    
    if kind == 'time':
        return gettime(d['meta']['term_of_imprisonment'])
    
    return label
    


In [4]:
from sklearn.externals import joblib
import pickle

dim = 300

def cut_text(alltext):
    train_text = []
    for text in alltext:
        train_text.append(' '.join(jieba.cut(text)))
     
    return train_text

def read_trainData(path):
    fin = open(path, 'r', encoding = 'utf8')
    
    alltext = []
    
    accu_label = []
    law_label = []
    time_label = []

    line = fin.readline()
    while line:
        d = json.loads(line)
        alltext.append(d['fact'])
        accu_label.append(getlabel(d, 'accu'))
        law_label.append(getlabel(d, 'law'))
        time_label.append(getlabel(d, 'time'))
        line = fin.readline()
    fin.close()

    return alltext, accu_label, law_label, time_label

In [6]:
###############################################测试部分###########################################################

In [5]:
from keras.preprocessing.text import Tokenizer
texts = ['a a a', 'b b', 'c']
tokenizer = Tokenizer(num_words=2)
tokenizer.fit_on_texts(texts)
tokenizer.word_index
tokenizer.texts_to_sequences(texts)


[[1, 1, 1], [], []]

In [6]:
texts2 = ['m m m', 'b']
tokenizer.fit_on_texts(texts2)

In [ ]:
tokenizer.fit_on_

In [20]:
text3 = ['n n', 'b', 'a a']

In [21]:
tokenizer.texts_to_sequences(text3)

[[], [], [1, 1]]

In [26]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer2 = pickle.load(handle)

In [5]:
tokenizer2.texts_to_sequences(texts2)

In [9]:
tokenizer.word_index

{'a': 1, 'b': 2, 'c': 4, 'm': 3}

In [ ]:
tokenizer.texts_to_sequences(texts)

In [ ]:
tokenizer.texts_to_sequences(['i a m m'])

In [ ]:
print("Found %s unique tokens." % len(tokenizer.word_index))

In [ ]:
#######################################测试部分结束——不用运行######################################################

In [28]:
class DataProcess(object):

    def __init__(self, path, data_path, max_sequence_length = 250, embedding_dim=1000):
        self.path = path                # 数据路径
        self.data_path = data_path
        self.max_sequence_length = max_sequence_length   # 最长文本长度, 每个fact有多少个word_index [1, 23,,...],多出的补零
        
        self.embedding_dim = embedding_dim
        self.embedding_index = {}           # 单词与嵌入向量映射 [word +vec] 
        self.embedding_matrix = None    # 嵌入矩阵 [word_index]
        self.word_index = {}   # 单词索引
        
        self.alltext = None     # 训练facts        
        self.accu_label = None    #罪行标签
        self.law_label = None     #发条标签
        self.time_label = None    #判刑标签        
        self.texts = [] #分词后的训练样本

        self.converTextToTensor(self.path + 'predictor/model/tokenizer.pickle', self.path +'word_index.pkl')
        self.processEmbedding(self.path+'word_index.pkl', self.path +'embedding_matrix.pkl', self.path+'w2v.pkl')

    #format our text samples and labels into tensors that can be fed into a neural network.
    def converTextToTensor(self, tokenizer_path = './model/tokenizer.pickle',
                           word_index_path= './word_index.pkl'):
        print('call member convertTextToTensor')
        self.alltext, self.accu_label, self.law_label, self.time_label = read_trainData(data_path)
        self.texts = cut_text(self.alltext)  # [['','',''], ['','','']]
            
        tokenizer = Tokenizer(num_words=10000) #出现次数top10000才会用来index
        tokenizer.fit_on_texts(self.texts)
        with open(tokenizer_path, 'wb') as handle:
            pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
            
        self.word_index = tokenizer.word_index  #{word:index,...}，索引index从1开始
        print("Found %s unique tokens." % len(self.word_index))
        with open(word_index_path, 'wb') as f:
            pickle.dump(self.word_index,f, -1)
        f.close()
        
        
        sequence = tokenizer.texts_to_sequences(self.texts)  # 将文本转换成整数序列            
        self.texts = pad_sequences(sequence, maxlen=self.max_sequence_length, padding= 'post', truncating = 'post')
        #[[1, 1, 3], [2, 1, 0]] #此为输入
        print('pad done')

    #compute the embedding and mapping the words/word_index to known embeddings
    def processEmbedding(self, word_index_path = './word_index.pkl', 
                         embedding_path = './embedding_matrix.pkl', word2vec_path = './w2v.model'):
        print('call processEmbedding')
        if os.path.isfile(embedding_path):
            print('load embedding matrix')
            with open(embedding_path, 'rb') as f:
                self.embedding_matrix = pickle.load(f)
        else:
            print('calculate embedding matrix')
            facts = self.alltext
            sentences = [[word for word in fact.split()] for fact in facts]
            word_model = gensim.models.Word2Vec(sentences, size= self.embedding_dim, min_count = 10, window = 5)
            word_model.save(word2vec_path)
            joblib.dump(word_model, './predictor/model/word_model2.model')
#             print('saving word embeddingmodel')
            
#             word_model = gensim.models.Word2Vec.load(word2vec_path) 
#             word_model2 = joblib.load('./predictor/model/word_model2.model')
#             print('word_model built completed.')

            with open(word_index_path, 'rb') as f:
                dic = pickle.load(f)
                self.embedding_matrix = np.zeros((len(self.word_index) + 1, self.embedding_dim))
                for word, i in dic.items():
                    try:
                        embedding_vector = word_model[word]
                    except:
                        embedding_vector = None
                    if embedding_vector is not None:
                        self.embedding_matrix[i] = embedding_vector 
                        # self.embedding_index[word] = word_model[word]                         
            print('Result embedding shape:', self.embedding_matrix.shape)
            
            with open(embedding_path, 'wb') as f:
                pickle.dump(self.embedding_matrix, f, -1)          
            print("Found %s word vectors." % len(self.embedding_matrix))
#             with open(path_to_save, 'wb') as f:
#                 pickle.dump(self.embeddings_index, f)
#         self.max_nb_words = len(self.embedding_matrix)

#     def word2idx(word):
#         return word_model.wv.vocab[word].index
#     def idx2word(idx):
#         return word_model.wv.index2word[idx]


In [ ]:
if os.path.isfile(embedding_path):
    print('load embedding matrix')
    with open(embedding_path, 'rb') as f:
        self.embedding_matrix = pickle.load(f)

In [26]:
def getCNN(max_sequence_length, word_index, embedding_dim, embedding_matrix, filter_sizes = [3,4,5], outpu_dim=1):

    print('call getCNN')
    convs = []
    # 输入层次
    sequence_input = Input(shape = (max_sequence_length,), dtype= 'int32', name = 'input')

    # 嵌入层
    embedding_layer = Embedding(len(word_index)+1,
                                embedding_dim,
                                weights= [embedding_matrix],
                                input_length= max_sequence_length,
                                trainable= False)
    embedded_sequence = embedding_layer(sequence_input)

    for fsz in filter_sizes:
        conv = Conv1D(32, kernel_size= fsz, activation='relu')(embedded_sequence)
        pool = MaxPooling1D(2)(conv)
        convs.append(pool)
    merge1 = concatenate(convs, axis=1)
    conv1 = Conv1D(32,4, activation='relu')(merge1)
    pool1 = MaxPooling1D(5)(conv1)
    flat = Flatten()(pool1)
    dense = Dense(32,activation='relu')(flat)
    preds = Dense(output_dim, activation='sigmoid', name='pred')(dense)

    model = Model(inputs=sequence_input,outputs=preds)

    return model

def generate_batch_data(input, output, batch_size):

    i = 0
    while 1:
        yield (input[i:i+batch_size, :],np.array(output[i: i+batch_size,:].todense()))
        i += batch_size
        if (i+ batch_size) > input.shape[0]:
            i = 0


def validaton_generator(vali_input, vali_output, batch_size):
    i = 0
    while 1:
        yield (vali_input[i:i + batch_size, :], np.array(vali_output[i: i + batch_size, :].todense()))
        i += batch_size
        if (i + batch_size) > vali_input.shape[0]:
            i = 0

def train_CNN(vecs, label, batch_size = 2000):            
    #  模型学习
    # 构建一个CNN模型
    model = getCNN(dataSet.max_sequence_length,
                   dataSet.word_index,
                   dataSet.embedding_dim,
                   dataSet.embedding_matrix)

    print('compile')
    # 编译CNN模型
    if learner.lower() == "adagrad":
        model.compile(optimizer=Adagrad(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    elif learner.lower() == "rmsprop":
        model.compile(optimizer=RMSprop(lr=learning_rate), loss='binary_crossentropy',metrics= ['accuracy'])
    elif learner.lower() == "adam":
        model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

    else:
        model.compile(optimizer=SGD(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    print(model.summary())
    early_stopping = EarlyStopping(monitor='val_loss', patience=2)
    accu_model = model.fit_generator(generate_batch_data(np.array(vecs)[0:10000,:],label[0:10000,:], batch_size),
                               steps_per_epoch=np.array(vecs)[0:10000,:].shape[0] / batch_size , epochs=1,
                               callbacks=[early_stopping])

In [ ]:
if __name__ == '__main__':
    max_sequence_length = 250
    embedding_dim = 2000
    output_dim = 1 # 先做单模型的

    #CNN模型超参
    learner = 'rmsprop'
    learning_rate = 1e-3
    batch_size = 2000

    # 构造数据集
    path = './'
    data_path = './CAIL_data/data_train.json'
    dataSet = DataProcess(path, data_path, max_sequence_length, embedding_dim)
 
    

    print('starting fitting')

    print('accu CNN')
    accu = train_CNN(dataSet.texts, dataSet.accu_label, batch_size)
    print('law CNN')
    law = train_CNN(dataSet.texts, dataSet.law_label, batch_size)
    print('time CNN')
    time = train_SVC(dataSet.texts, dataSet.time_label, batch_size)


    joblib.dump(accu, 'predictor/model/accu.model')
    joblib.dump(law, 'predictor/model/law.model')
    joblib.dump(time, 'predictor/model/time.model')
    
    # 模型指标
    print('fit done')
    for key, value in hist.history.items():
        print(key, value)
    # 保存模型
    model.save('model1.h5')


call member convertTextToTensor


In [ ]:
#########################################预测部分#################################################################

In [ ]:
# class testDataProcess(object):

#     def __init__(self, content, max_sequence_length = 250, embedding_dim = 1000, 
#                  word_index_path = './word_index.pkl', 
#                  embedding_matrix = './embedding_matrix.pkl'):
        
#         self.max_sequence_length = max_sequence_length   # 最长文本长度, 每个fact有多少个word_index [1, 23,,...],多出的补零
#         self.embedding_matrix = None    # 嵌入矩阵 [word_index]
#         self.word_index = {}   # 单词索引
#         self.embedding_dim = None
#         self.alltext = content    # 训练facts            
#         self.texts = [] #分词后的训练样本

#         self.converTextToTensor(self.path +'word_index.pkl')

#     def converTextToTensor(self, word_index_path= './tokenizer.pickle.pkl'):
#         print('load tokenizer')
#         with open('tokenizer.pickle', 'rb') as handle:
#             tokenizer = pickle.load(handle)
            
#         self.texts = cut_text(self.alltext)  # [['','',''], ['','','']]
            
#         sequence = tokenizer.texts_to_sequences(self.texts)  # 将文本转换成整数序列            
#         self.texts = pad_sequences(sequence, maxlen=self.max_sequence_length, padding= 'post', truncating = 'post')
#         #[[1, 1, 3], [2, 1, 0]] #此为输入
#         print('pad done')

  

In [ ]:
import json
import jieba
from sklearn.externals import joblib



class Predictor(object):
    def __init__(self):
        with open('predicotr/model/tokenizer.pickle', 'rb') as handle:
            self.tokenizer = pickle.load(handle)
        self.law = joblib.load('predictor/model/law.model')
        self.accu = joblib.load('predictor/model/accu.model')
        self.time = joblib.load('predictor/model/time.model')
        self.batch_size = 1

    def predict_law(self, vec):
        y = self.law.predict(vec)
        return [y[0] + 1]
    
    def predict_accu(self, vec):
        y = self.accu.predict(vec)
        return [y[0] + 1]
    
    def predict_time(self, vec):

        y = self.time.predict(vec)[0]
        
        #返回每一个罪名区间的中位数
        if y == 0:
            return -2
        if y == 1:
            return -1
        if y == 2:
            return 120
        if y == 3:
            return 102
        if y == 4:
            return 72
        if y == 5:
            return 48
        if y == 6:
            return 30
        if y == 7:
            return 18
        else:
            return 6
    
    
    def predict(self, content):

            
        self.texts = cut_text(self.alltext)  # [['','',''], ['','','']]
            
        sequence = self.tokenizer.texts_to_sequences(self.texts)  # 将文本转换成整数序列            
        self.texts = pad_sequences(sequence, maxlen=self.max_sequence_length, padding= 'post', truncating = 'post')
        #[[1, 1, 3], [2, 1, 0]] #此为输入
        print('pad done')

        vec = self.texts
        ans = {}

        ans['accusation'] = self.predict_accu(vec)
        ans['articles'] = self.predict_law(vec)
        ans['imprisonment'] = self.predict_time(vec)
        
        #print(ans)
        return [ans]


In [ ]:
import json
import os
import multiprocessing

#from predictor import Predictor

data_path = "./input/"  # The directory of the input data
output_path = "./output/"  # The directory of the output data


def format_result(result):
    rex = {"accusation": [], "articles": [], "imprisonment": -3}

    res_acc = []
    for x in result["accusation"]:
        if not (x is None):
            res_acc.append(int(x))
    rex["accusation"] = res_acc

    if not (result["imprisonment"] is None):
        rex["imprisonment"] = int(result["imprisonment"])
    else:
        rex["imprisonment"] = -3

    res_art = []
    for x in result["articles"]:
        if not (x is None):
            res_art.append(int(x))
    rex["articles"] = res_art

    return rex


if __name__ == "__main__":
    user = Predictor()
    cnt = 0


    def get_batch():
        v = user.batch_size
        if not (type(v) is int) or v <= 0:
            raise NotImplementedError

        return v


    def solve(fact):
        result = user.predict(fact)

        for a in range(0, len(result)):
            result[a] = format_result(result[a])

        return result


    for file_name in os.listdir(data_path):
        inf = open(os.path.join(data_path, file_name), "r")
        ouf = open(os.path.join(output_path, file_name), "w")

        fact = []

        for line in inf:
            fact.append(json.loads(line)["fact"])
            if len(fact) == get_batch():
                result = solve(fact)
                cnt += len(result)
                for x in result:
                    print(json.dumps(x), file=ouf)
                fact = []

        if len(fact) != 0:
            result = solve(fact)
            cnt += len(result)
            for x in result:
                print(json.dumps(x), file=ouf)
            fact = []

        ouf.close()
